# 사전 준비: 샘플 Agent 생성

## 개요

평가할 agent를 먼저 생성해보겠습니다. 이 튜토리얼에서는 서로 다른 프레임워크를 사용하여 두 개의 샘플 agent를 생성합니다:
- [Strands Agents SDK](https://strandsagents.com/)
- [LangGraph](https://www.langchain.com/langgraph)

두 agent 모두 Amazon Bedrock의 Anthropic Claude Haiku 4.5를 LLM model로 사용하지만, 원하는 다른 model을 사용할 수도 있으며 동일한 기능을 제공합니다:
- **Math Tool**: 기본 수학 연산을 수행하는 tool
- **Weather Tool**: 날씨 tool의 더미 구현


아키텍처는 다음과 같습니다:

![Architecture](../images/agent_architecture.png)

## 사전 준비 사항
- Python 3.10+
- AWS credentials

In [ ]:
!pip install -r ../requirements.txt

## 설정

필요한 패키지를 import하고 AWS 세션을 구성합니다:

In [ ]:
# AgentCore Runtime 및 AWS 세션 관리를 위한 라이브러리 import
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import uuid
import time

# AWS 세션 생성 및 리전 정보 가져오기
boto_session = Session()
region = boto_session.region_name
print(f"Using region: {region}")

## Strands Agent 배포
Strands Agent를 AgentCore Runtime에 배포해보겠습니다

In [ ]:
# AgentCore Runtime 인스턴스 생성
agentcore_runtime = Runtime()

agent_name = "ac_eval_strands2"
# Strands agent 설정: entrypoint, 실행 role, ECR 자동 생성 등
response = agentcore_runtime.configure(
    entrypoint="eval_agent_strands.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements_strands.txt",
    region=region,
    agent_name=agent_name,
    idle_timeout=120  # 유휴 시간 초과 설정 (초)
)
# Agent 배포 시작
launch_result_strands = agentcore_runtime.launch()
print(f"Strands agent deployment started: {launch_result_strands}")

### AgentCore Runtime의 배포 상태 확인
배포가 ACTIVE 상태가 될 때까지 기다립니다..

In [ ]:
def wait_for_deployment(runtime, name):
    # 배포 완료 상태 목록
    end_statuses = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
    while True:
        status_response = runtime.status()
        status = status_response.endpoint['status']
        print(f"{name} status: {status}")
        # 배포가 완료 상태에 도달하면 종료
        if status in end_statuses:
            print(f"{name} deployment completed with status: {status}")
            return status
        time.sleep(10)  # 10초마다 상태 확인

strands_status = wait_for_deployment(agentcore_runtime, "Strands")

### Runtime에서 Strands Agent 호출
페이로드를 사용하여 AgentCore Runtime 엔드포인트를 호출해 Strands agent를 테스트해보겠습니다.

In [ ]:
# 세션 추적을 위한 고유 ID 생성
session_id_strands = str(uuid.uuid4())
print(f"Session ID: {session_id_strands}")

In [ ]:
# Math tool 테스트: 기본 수학 연산 수행
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "How much is 2+2?"},
    session_id=session_id_strands
)
invoke_response

In [ ]:
# Weather tool 테스트: 날씨 정보 조회
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "How is the weather now?"},
    session_id=session_id_strands
)
invoke_response

In [ ]:
# Tool 없이 LLM 지식만으로 답변 가능한 질문 테스트
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "Can you tell me the capital of the US?"},
    session_id=session_id_strands
)
invoke_response

## LangGraph Agent를 AgentCore Runtime에 배포

LangGraph agent도 AgentCore Runtime에 배포해보겠습니다.

In [ ]:
# LangGraph agent용 Runtime 인스턴스 생성
langgraph_agentcore_runtime = Runtime()
agent_name = "ac_eval_langgraph2"
# LangGraph agent 설정 및 배포
response = langgraph_agentcore_runtime.configure(
    entrypoint="eval_agent_langgraph.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements_langgraph.txt",
    region=region,
    agent_name=agent_name,
    idle_timeout=120
)
launch_result_langgraph = langgraph_agentcore_runtime.launch()
print(f"LangGraph agent deployment started: {launch_result_langgraph}")

### LangGraph agent의 상태 확인
LangGraph agent를 AgentCore Runtime에 배포했으니, 배포 상태를 확인해보겠습니다

In [ ]:
langgraph_status = wait_for_deployment(langgraph_agentcore_runtime, "LangGraph")

### Runtime에서 Langgraph Agent 호출
페이로드를 사용하여 AgentCore Runtime의 LangGraph agent 엔드포인트를 테스트합니다:

In [ ]:
# LangGraph agent용 세션 ID 생성
session_id_langgraph = str(uuid.uuid4())
print(f"Session ID: {session_id_langgraph}")

In [ ]:
response = langgraph_agentcore_runtime.invoke(
    payload={"prompt": "What is 2+2?"},
    session_id=session_id_langgraph
)
print(response)

In [ ]:
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "What is the weather now?"},
    session_id=session_id_langgraph
)
invoke_response

In [ ]:
invoke_response = agentcore_runtime.invoke(
    payload={"prompt": "Can you tell me the capital of the US?"},
    session_id=session_id_langgraph
)
invoke_response

In [ ]:
print(f"Strands: {launch_result_strands}, Session: {session_id_strands}")
print(f"LangGraph: {launch_result_langgraph}, Session: {session_id_langgraph}")

In [ ]:
# %store magic command: 변수를 저장하여 다른 노트북에서 재사용 가능
%store launch_result_strands
%store session_id_strands
%store launch_result_langgraph
%store session_id_langgraph

## 다음 단계

이제 필요한 모든 사전 준비 사항이 완료되었으니, 개별 evaluation 튜토리얼을 진행해보겠습니다:
evaluation 튜토리얼을 계속 진행하세요:
- [01-creating-custom-evaluators](../01-creating-custom-evaluators/): 커스텀 evaluator 생성
- [02-running-evaluations](../02-running-evaluations/): 온디맨드 및 온라인 evaluation 실행
- [03-advanced](../03-advanced/): 고급 기술 및 대시보드